In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import requests
import selenium

In [2]:
import pymongo

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.March_tweets
collection = db.trends
collection2 = db.tweets

In [7]:
path = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(path)

In [11]:
driver.get("https://us.trend-calendar.com/trend/2020-03-01.html")

In [9]:
trend_dates = {}
pageable = True
counter = 31
while pageable == True:
    time.sleep(0.5)
    contents = driver.find_element_by_class_name("ranking")
    contents_html = contents.get_attribute("innerHTML")
    soup = bs(contents_html)
    anchors = soup.find_all("a")
    trends = []
    header = driver.find_element_by_class_name("entry-title")
    date = header.text.split("on ",1)[1]
    for a in anchors[:3]:
        url = a["href"]
        text = a.text
        d = {"URL": url, "trends": text}
        trends.append(d)
    trend_dates[date] = trends
    try:
        submit_button = driver.find_elements_by_class_name("yesterdaytomorrow")[1]
        pageable = True
        submit_button.click()
        counter -= 1
        if counter == 0:
            pageable = False
    except:
        pageable = False
collection.insert_one(trend_dates)

In [ ]:
trend_dates

In [20]:
driver.quit()

In [12]:
driver.get("https://www.twitter.com/login")
time.sleep(1)
usernameInput = driver.find_element_by_name("session[username_or_email]")
passwordInput = driver.find_element_by_name("session[password]")
usernameInput.send_keys("chuck45630256")
passwordInput.send_keys("Longbow1@")
passwordInput.send_keys(Keys.ENTER)
time.sleep(5)

In [18]:
March = []
March_dict = {}
for date in trend_dates:
    trend_dict = {}
    for trend in trend_dates[date]:
        driver.get(trend['URL'])
        time.sleep(1.5)
        counter = 3
        text_tweets = []
        while counter != 0:
            try:
                time.sleep(2)
                contents = driver.find_element_by_class_name("css-1dbjc4n")
                contents_html = contents.get_attribute("innerHTML")
                soup = bs(contents_html)
                tweet_html = soup.find_all("div", {"data-testid":"tweet"})
                for tweet in tweet_html:
                    text = tweet.find("div", class_ = "css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0").get_text()
                    text_tweets.append(text)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
                counter -= 1
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
                counter -= 1
            trend_dict[trend['trends']] = text_tweets
        March.append(text_tweets)
        time.sleep(1)
    March_dict[date]={"Tweet":trend_dict}

TypeError: 'ObjectId' object is not iterable

In [ ]:
March_dict

In [ ]:
March_dict

In [19]:
collection2.insert_one(March_dict)

In [ ]:
file1 = open("March tweets separated by day text.txt",encoding="utf8")

In [ ]:
data = file1.readlines()

In [ ]:
data

In [ ]:
chunks = [March[x:x+3] for x in range(0, len(March), 3)]

In [ ]:
type(chunks)

In [ ]:
with open('March tweets separated by day text.csv', 'w', encoding="utf-8") as filehandle:
    for listitem in chunks:
        filehandle.write('%s\n' % listitem)

In [ ]:
len(chunks)

In [ ]:
len(chunks[0][0])